In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_email = pd.read_csv('/content/drive/MyDrive/spam_ham_dataset.csv')

In [ ]:
df_email.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [ ]:
###Drop Nan Values
df_email=df_email.dropna()
df_email.reset_index(inplace=True)

In [ ]:
import re
import string
def cleaning(txt):
    # case folding
    text = txt.lower()
    # remove multiple space, tabs, dan newlines
    text = re.sub('\s+',' ',text)
    # remove links
    text = text.replace("http://", " ").replace("https://", " ")
    # remove special characters
    text = text.encode('ascii', 'replace').decode('ascii')
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove punctuation
    text = ''.join([word for word in text if word not in string.punctuation])
    #remove single character
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    #remove numbers
    text = re.sub(r"\d+", "", text)
    #remove multiple spaces (again)
    text = re.sub('\s+',' ',text)
    return text

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem(email):
    email = email.split()
    email = [ps.stem(word) for word in email if not word in stopwords.words('english')]
    email = ' '.join(email)
    return email

In [ ]:
#Data Preprocessing
df_email['text'] = df_email['text'].apply(lambda x: cleaning(x))

In [ ]:
df_email['text'] = df_email['text'].apply(lambda x: stem(x))

In [ ]:
df_email.head()

,index,Unnamed: 0,label,text,label_num
0,0,605,ham,subject enron methanol meter follow note gave ...,0
1,1,2349,ham,subject hpl nom januari see attach file hplnol...,0
2,2,3624,ham,subject neon retreat ho ho ho around wonder ti...,0
3,3,4685,spam,subject photoshop window offic cheap main tren...,1
4,4,2030,ham,subject indian spring deal book teco pvr reven...,0


In [ ]:
## Get the Independent Features
X=df_email.drop(['index','label_num','label','Unnamed: 0'],axis=1)

In [ ]:
## Get the Dependent features
y=df_email['label_num']

In [ ]:
X.shape

(5171, 1)

In [ ]:
X

,text
0,subject enron methanol meter follow note gave ...
1,subject hpl nom januari see attach file hplnol...
2,subject neon retreat ho ho ho around wonder ti...
3,subject photoshop window offic cheap main tren...
4,subject indian spring deal book teco pvr reven...
...,...
5166,subject put ft transport volum decreas contrac...
5167,subject follow nom hpl take extra mmcf weekend...
5168,subject calpin daili ga nomin juli mention ear...
5169,subject industri worksheet august activ attach...


In [ ]:
y.shape

(5171,)

In [ ]:
y

0       0
1       0
2       0
3       1
4       0
       ..
5166    0
5167    0
5168    0
5169    0
5170    1
Name: label_num, Length: 5171, dtype: int64

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=10000

One Hot Representation

In [ ]:
emails=X.copy()

In [ ]:
emails

,text
0,subject enron methanol meter follow note gave ...
1,subject hpl nom januari see attach file hplnol...
2,subject neon retreat ho ho ho around wonder ti...
3,subject photoshop window offic cheap main tren...
4,subject indian spring deal book teco pvr reven...
...,...
5166,subject put ft transport volum decreas contrac...
5167,subject follow nom hpl take extra mmcf weekend...
5168,subject calpin daili ga nomin juli mention ear...
5169,subject industri worksheet august activ attach...


In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in emails.text] 

Embedding Representation

In [ ]:
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 8088 5470  740]
 [   0    0    0 ... 7009 5124 7009]
 [4655 1815 4394 ... 1735 9936 3878]
 ...
 [   0    0    0 ... 1723 5114 1511]
 [   0    0    0 ... 2175 5189 1809]
 [9885  924   10 ... 8447 1108 8370]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0, 8669, 3111, 8861,
       4979, 9861, 9714,  630, 3087, 9352, 2979, 1809, 5189, 6574, 4160,
       8173, 9581, 6967, 4706, 7793, 9756, 7032, 6967, 6818, 9793, 1723,
       6244, 9942, 7375, 8088, 5470,  740], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 40)            400000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(5171, (5171,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((5171, 50), (5171,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Train Model

In [ ]:
### Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
55/55 [==============================] - 7s 80ms/step - loss: 0.4102 - accuracy: 0.8014 - val_loss: 0.1996 - val_accuracy: 0.9145
Epoch 2/10
55/55 [==============================] - 4s 70ms/step - loss: 0.1366 - accuracy: 0.9550 - val_loss: 0.1142 - val_accuracy: 0.9637
Epoch 3/10
55/55 [==============================] - 4s 70ms/step - loss: 0.0478 - accuracy: 0.9859 - val_loss: 0.0896 - val_accuracy: 0.9619
Epoch 4/10
55/55 [==============================] - 4s 69ms/step - loss: 0.0313 - accuracy: 0.9922 - val_loss: 0.0907 - val_accuracy: 0.9660
Epoch 5/10
55/55 [==============================] - 4s 71ms/step - loss: 0.0269 - accuracy: 0.9928 - val_loss: 0.1021 - val_accuracy: 0.9654
Epoch 6/10
55/55 [==============================] - 4s 70ms/step - loss: 0.0135 - accuracy: 0.9980 - val_loss: 0.0879 - val_accuracy: 0.9766
Epoch 7/10
55/55 [==============================] - 4s 70ms/step - loss: 0.0073 - accuracy: 0.9986 - val_loss: 0.0925 - val_accuracy: 0.9795
Epoch 8/10
55

Performance Metrics And Accuracy

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9619214997070885

0.9619214997070885